In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v6 import CNN_v6

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v6 = CNN_v6((480, 640, 4), split_into_parts=20)
cnn_v6.def_model()

In [5]:
images = cnn_v6._prepare_images(image_train)
anwsers = cnn_v6._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v6.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v6/weights/cnn_v6_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str, epoch: int = 0):
        self.__path = path
        self.__current_epoch = epoch
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path, 5)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 346s 278us/step - loss: 249.7452 - accuracy: 0.8236 - val_loss: 77.3207 - val_accuracy: 0.8438
Epoch 2/5
1244460/1244460 [==============================] - 339s 272us/step - loss: 65.8641 - accuracy: 0.8977 - val_loss: 38.5176 - val_accuracy: 0.8481
Epoch 3/5
1244460/1244460 [==============================] - 338s 271us/step - loss: 28.1668 - accuracy: 0.8982 - val_loss: 17.5020 - val_accuracy: 0.8533
Epoch 4/5
1244460/1244460 [==============================] - 337s 270us/step - loss: 12.8268 - accuracy: 0.8929 - val_loss: 8.2077 - val_accuracy: 0.8522
Epoch 5/5
1244460/1244460 [==============================] - 335s 269us/step - loss: 5.8596 - accuracy: 0.8843 - val_loss: 3.8361 - val_accuracy: 0.8497


In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=5000, epochs=10,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/10
1244460/1244460 [==============================] - 346s 278us/step - loss: 242.9190 - accuracy: 0.8206 - val_loss: 78.3556 - val_accuracy: 0.6528
Epoch 2/10
1244460/1244460 [==============================] - 343s 275us/step - loss: 65.7822 - accuracy: 0.8951 - val_loss: 40.1079 - val_accuracy: 0.8533
Epoch 3/10
1244460/1244460 [==============================] - 338s 272us/step - loss: 29.6050 - accuracy: 0.8980 - val_loss: 18.9682 - val_accuracy: 0.8562
Epoch 4/10
1244460/1244460 [==============================] - 339s 272us/step - loss: 13.7836 - accuracy: 0.8952 - val_loss: 8.6607 - val_accuracy: 0.8629
Epoch 5/10
1244460/1244460 [==============================] - 337s 270us/step - loss: 6.2930 - accuracy: 0.8912 - val_loss: 3.9919 - val_accuracy: 0.8656
Epoch 6/10
1244460/1244460 [==============================] - 337s 271us/step - loss: 2.9797 - accuracy: 0.8905 - val_loss: 2.3374 - val_accuracy: 0.8731
Epoch 7/10
12444